In [ ]:
import openai
import time
import pandas as pd

In [ ]:
client = openai.Client()

Realiza o upload de um arquivo para carregar informações ao modelo

In [ ]:
file = client.files.create(
    file=open("data_upload.txt", "rb"),
    purpose="assistants"
)

Cria o assistente para acessar o arquivo

In [ ]:

assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas.",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4.0"
)


Função: Cria uma thread de conversa e envia a pergunta como mensagem.

In [ ]:
pergunta = "Qual o rating médio das vendas do supermercado"
#pergunta = "Gere um grafico de vendas por linha de produtos"
#pergunta = (
#    "Gere 15 perguntas para conversação avançada
#    em ingles que nao estejam contidas no arquivo data_upload
#    sobre os dados que ainda não estão presentes no arquivo. Nao repita as questões ou 
#    qualquer pergunta ou frase que já esteja contida nele."
#)


# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta)

In [ ]:
#Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [ ]:
# Aguarda a thread rodar
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [ ]:
# Verifica a resposta quando tivermos a resposta da thread
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens.data[0].content[0].text.value)
else:
    print(f"Erro {run.status}")

In [ ]:
# por hora esse print ficara commitado porque ja esta sendo considerado
# mais abaixo
# print(mensagens.data[0].content[0].text.value)

In [ ]:
#analisando a logica/passos do modelo para chegar na resposta
run_steps = client.beta.threads.run.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [ ]:
for step in run_steps.data[::-1]:
    print(f"\n===Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("-" * 10)
            print(tool_call.code_interpreter.input)
            print("-" * 10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
        thread_id=thread.id,
        message_id=step.step_details.message_creation.message_id
    )

# print(message.content[0].text.value)
# comentado acim apara quando a geração do output é grafico e não texto(conforme a pergunta)

In [ ]:

#condicional adaptado para quando a pergunta solicita texto ou grafico
#Função: Verifica se a resposta é texto ou imagem. Se for imagem, salva localmente.
if message.content[0].type == "text":
    print(message.content[0].text.value)
if message.content[0].type == "image_file":
    file_id = message.content[0].image_file.file_id
    image_data = client.files.content(file_id)
    with open(f"files/{file.id}.png", "wb") as f:
        f.write(image_data.read())
    print(f"Imagem {file_id} salva")
